Ratatui is a crate for building terminal user interfaces in Rust.

One of the unique features of Ratatui is that it is an immediate mode rendering library.
In this post, I'm going to describe some of the primitives of Ratatui.
I rely on the concepts described in this post in every Ratatui application I build.

## Immediate Mode Rendering

User interfaces can broadly be classified into two kinds:

- immediate mode GUIs,
- retained mode GUIs.

Casey Muratori has a great video on immediate mode rendering. 

{{< video https://www.youtube.com/watch?v=Z1qyvQsjK5Y >}}

At a very high level, in retained mode GUIs, you create UI elements and pass it to a framework and the framework is in charge of displaying them. For example, you can create a text field and input field, and then the browser will render them. The browser is in charge of handling events, and as a developer you have to define how these events interact with these widgets.

For example, in a simple counter example in a browser, we have to set up an `incrementCounter` and `decrementCounter` callbacks that update the relevant element's state. The browser is responsible for displaying these elements, receiving user inputs, calling the appropriate `onclick` callback, etc.

In [2]:
//| code-fold: true

fn show_html<D>(content: D) where D: std::fmt::Display {
    println!(r#"EVCXR_BEGIN_CONTENT text/html
<div style="display: flex; justify-content:start; gap: 1em; margin: 1em">
{}
</div>
EVCXR_END_CONTENT"#, content);
}

show_html(r#"
<text> Counter: </text>
<text id="counter">0</text>

<button onclick="incrementCounter()">Increment</button>
<button onclick="decrementCounter()">Decrement</button>

<script>
    var counterElement = document.getElementById("counter");

    var counterValue = 0;
    counterElement.textContent = counterValue;

    function incrementCounter() {
        counterValue++;
        counterElement.textContent = counterValue;
    }

    function decrementCounter() {
        counterValue--;
        counterElement.textContent = counterValue;
    }
</script>
"#)

()

In immediate mode rendering, however, _you_ are responsible for rendering the UI every "frame".
This is typically done in a `for` loop or a `while true` loop in your application; and you use an immediate mode rendering library (in our case `ratatui`) to render the elements.
This means you as the developer of the application using immediate mode rendering are responsible for a lot more things but it also gives you more control and freedom.

## Buffer Primitives

For this post, we are going to use 2 dependencies, `ratatui` and `ratatui-macros`.

In [5]:
//| code-fold: true
:dep ratatui = "0.26.2"
:dep ratatui-macros = "0.4.0"

One of Ratatui's core primitives is a `Rect` struct. Let's create one:

In [17]:
use ratatui::layout::Rect;

let (x, y, width, height) = (0, 0, 80, 5);
let area = Rect::new(x, y, width, height);
area

Rect { x: 0, y: 0, width: 80, height: 5 }

In Ratatui, every "widget" renders into a `Buffer` of a fixed size that is equal to the terminal dimensions. 
Let create an empty buffer:

In [19]:
use ratatui::buffer::Buffer;

let mut buf = Buffer::empty(area);
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "                                                                                ",
        "                                                                                ",
        "                                                                                ",
        "                                                                                ",
        "                                                                                ",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

We can render into the buffer once by using the `Block` widget with a border.

In [20]:
use ratatui::widgets::Block;
use ratatui::widgets::Widget; // required trait for `.render` method

let block = Block::bordered();
block.render(area, &mut buf);

In [21]:
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "┌──────────────────────────────────────────────────────────────────────────────┐",
        "│                                                                              │",
        "│                                                                              │",
        "│                                                                              │",
        "└──────────────────────────────────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

Let's also add a title.

In [22]:
let block = Block::bordered().title("Counter Example");
block.render(area, &mut buf);

In [23]:
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "┌Counter Example───────────────────────────────────────────────────────────────┐",
        "│                                                                              │",
        "│                                                                              │",
        "│                                                                              │",
        "└──────────────────────────────────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

Now let's render put text into the center of the buffer.

Let's say you have the following `App`:

In [24]:
#[derive(Debug, Default)]
pub struct App {
    counter: u8,
}
let mut app = App::default();
app

App { counter: 0 }

And you want to render the `App`'s `counter` in the center of the buffer.

In [25]:
use ratatui::widgets::Paragraph;

let inner_area = area.inner(&ratatui::layout::Margin { horizontal: 0, vertical: 2 });

let paragraph = Paragraph::new(format!("Counter: {}", app.counter)).centered();
paragraph.render(inner_area, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "┌Counter Example───────────────────────────────────────────────────────────────┐",
        "│                                                                              │",
        "│                                  Counter: 0                                  │",
        "│                                                                              │",
        "└──────────────────────────────────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

This is one frame of our UI!

Let's put our UI code into a function.

In [26]:
fn draw_ui(app: &App, area: Rect, buf: &mut Buffer) {
    let block = Block::bordered().title("Counter Example");
    block.render(area, buf);
    
    let inner_area = area.inner(&ratatui::layout::Margin { horizontal: 0, vertical: 2 });    
    let paragraph = Paragraph::new(format!("Counter: {}", app.counter)).centered();
    paragraph.render(inner_area, buf);
}

For the next frame, we can increment the counter and render into the buffer again.

In [27]:
app.counter += 1;

draw_ui(&app, area, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "┌Counter Example───────────────────────────────────────────────────────────────┐",
        "│                                                                              │",
        "│                                  Counter: 1                                  │",
        "│                                                                              │",
        "└──────────────────────────────────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

In [28]:
app.counter += 1;

draw_ui(&app, area, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "┌Counter Example───────────────────────────────────────────────────────────────┐",
        "│                                                                              │",
        "│                                  Counter: 2                                  │",
        "│                                                                              │",
        "└──────────────────────────────────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

In [29]:
app.counter += 1;

draw_ui(&app, area, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 80, height: 5 },
    content: [
        "┌Counter Example───────────────────────────────────────────────────────────────┐",
        "│                                                                              │",
        "│                                  Counter: 3                                  │",
        "│                                                                              │",
        "└──────────────────────────────────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

If we repeat this process of "updating state" and "drawing UI" in a loop, we get an immediate mode rendered UI.

Here's what a more complete counter application might look like with keyboard events.

![](./basic-app.webp)

If you are interested in seeing the full code regarding this, you can check out the [`basic-app`] tutorial on the Ratatui website.

[`basic-app`]: https://ratatui.rs/tutorials/counter-app/basic-app/

Ratatui uses a double buffer rendering technique that you can read about [here].

[here]: https://ratatui.rs/concepts/rendering/under-the-hood/

## Conclusion

In the next post, we'll discuss layout primitives.